In [3]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import gc
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split, KFold, StratifiedGroupKFold
from torch.nn.utils.rnn import pad_sequence
import cv2
import h5py
import io
import pandas.api.types
import random
import sklearn.metrics
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
import pandas.api.types
import sklearn.metrics
from math import sin,cos,pi
from sklearn.metrics import roc_auc_score, auc, roc_curve
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from transformers import get_cosine_schedule_with_warmup
from catboost import CatBoostClassifier, Pool, cv
from copy import deepcopy
import wandb
import timm 

In [5]:
timm.list_pretrained()[::-1]

['xcit_tiny_24_p16_384.fb_dist_in1k',
 'xcit_tiny_24_p16_224.fb_in1k',
 'xcit_tiny_24_p16_224.fb_dist_in1k',
 'xcit_tiny_24_p8_384.fb_dist_in1k',
 'xcit_tiny_24_p8_224.fb_in1k',
 'xcit_tiny_24_p8_224.fb_dist_in1k',
 'xcit_tiny_12_p16_384.fb_dist_in1k',
 'xcit_tiny_12_p16_224.fb_in1k',
 'xcit_tiny_12_p16_224.fb_dist_in1k',
 'xcit_tiny_12_p8_384.fb_dist_in1k',
 'xcit_tiny_12_p8_224.fb_in1k',
 'xcit_tiny_12_p8_224.fb_dist_in1k',
 'xcit_small_24_p16_384.fb_dist_in1k',
 'xcit_small_24_p16_224.fb_in1k',
 'xcit_small_24_p16_224.fb_dist_in1k',
 'xcit_small_24_p8_384.fb_dist_in1k',
 'xcit_small_24_p8_224.fb_in1k',
 'xcit_small_24_p8_224.fb_dist_in1k',
 'xcit_small_12_p16_384.fb_dist_in1k',
 'xcit_small_12_p16_224.fb_in1k',
 'xcit_small_12_p16_224.fb_dist_in1k',
 'xcit_small_12_p8_384.fb_dist_in1k',
 'xcit_small_12_p8_224.fb_in1k',
 'xcit_small_12_p8_224.fb_dist_in1k',
 'xcit_nano_12_p16_384.fb_dist_in1k',
 'xcit_nano_12_p16_224.fb_in1k',
 'xcit_nano_12_p16_224.fb_dist_in1k',
 'xcit_nano_12_p8_3

In [6]:
class CFG:
    class data:
        train_data= './isic-2024-challenge/train-metadata.csv'
        train_hdf5='./isic-2024-challenge/train-image.hdf5'
        output_prefix = 'effnet_res'
        num_workers = 8
        img_size = 384
        nfolds = 5
        positive_sampling = 5
        negs_per_positive = 10
        batch_size = 32
        eval_batch_size = 128
        seed = 56
    class model:
        model = 'efficientnetv2_rw_m.agc_in1k'
        pretrained = True
        optim = torch.optim.AdamW
        global_pool = 'avg' # 'avg', 'max', 'avgmax', 'catavg'
        drop_path_rate = 0.3 
        cls_drop = 0.2
        num_chanels = 3
        num_labels = 2
        hidden_size = 2152
        scheduler = 'cosine'
        max_epoches = 1
        lr = 2e-4
        num_cycles = 0.5
        warmup_ratio = 0.0
        lr_head = 1e-4
        eps = 1e-12
        weight_decay = 0.0
        weight_decay_head = 0.0
        betas = (0.9, 0.999)
    seed = 56
    gen_fold_number = 0
    fold_number = 0
    
class Transforms:
    train_pre_transforms = [A.Resize(CFG.data.img_size,CFG.data.img_size)]
    train_post_transforms = [
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0,p=1.0),
            ToTensorV2()
    ]
    
    transforms_train = A.Compose([
            A.Resize(CFG.data.img_size,CFG.data.img_size),
            A.RandomRotate90(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.ShiftScaleRotate(
                rotate_limit=(-10, 10),
                scale_limit=(-0.1,0.1),
                shift_limit=(-0.05, 0.05),
                p=0.25
            ),
            A.Downscale(p=0.25,scale_range=(0.25, 0.5)),
            A.RandomBrightnessContrast(brightness_limit=(0.2, -0.2), contrast_limit=(-0.2, 0.2),p=0.5),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
            ToTensorV2()
        ])
    transforms_val = A.Compose([
            A.Resize(CFG.data.img_size,CFG.data.img_size),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
            ToTensorV2()
        ])
    transforms_test = A.Compose([
            A.Resize(CFG.data.img_size,CFG.data.img_size),
            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0),
            ToTensorV2()
        ])

def set_wandb_cfg():
    config = {}
    for k,v in CFG.model.__dict__.items():
        if '__' not in k:
            config[k] = v
    for k,v in CFG.data.__dict__.items():
        if '__' not in k:
            config[k] = v
    config['fold_number'] = CFG.fold_number
    return config

In [7]:
def make_df(path):
    data = pd.read_csv(path)
    return data[['isic_id','target','patient_id']]

In [8]:
class PLDataset(Dataset):
    def __init__(self, df, transforms, reader, mode='train'):
        super().__init__()
        self.cfg = CFG.data
        self.data = df
        self.transforms = transforms
        self.hdf5_filles = reader
        self.mode = mode
    
    def get_transforms(self,image,augs):
        if augs == 'no' or self.mode != 'train':
            transforms = self.transforms.transforms_val
            return transforms(image=image)['image']
        else:
            transforms = A.Compose(
                self.transforms.train_pre_transforms + augs + self.transforms.train_post_transforms
            )
            return transforms(image=image)['image']
        
    def __getitem__(self, index):
        row = self.data.iloc[index]
        image = np.array(Image.open(io.BytesIO(self.hdf5_filles[row['isic_id']][()])))
        image = self.get_transforms(image,augs=row['augs_type'])
            
        return {
            'image': image.squeeze(0),
            'labels': row['target']
        }
    
    def __len__(self):
        return len(self.data)

In [9]:
class PLDataModule(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.data
        self.is_setup = False
        self.is_prepared = False
    
    def set_augs(self,df):
        augs = ['no'] * (df.shape[0] // self.cfg.positive_sampling)
        for i in range(df.shape[0] // self.cfg.positive_sampling, df.shape[0]):
            augs_lst = []
            if random.random() > 0.5:
                augs_lst.append(A.RandomRotate90(p=1.0))
            if random.random() > 0.5:
                augs_lst.append(A.HorizontalFlip(p=1.0))
            if random.random() > 0.5:
                augs_lst.append(A.VerticalFlip(p=1.0))
            if random.random() > 0.75:
                augs_lst.append(
                    A.ShiftScaleRotate(
                        rotate_limit=(-10, 10),
                        scale_limit=(-0.1,0.1),
                        shift_limit=(-0.05, 0.05),
                        p=1.0
                    )
                )
            if random.random() > 0.9:
                augs_lst.append(A.Downscale(p=1.0,scale_range=(0.25, 0.5)))
            if random.random() > 0.5:
                augs_lst.append(A.RandomBrightnessContrast(brightness_limit=(0.2, -0.2), contrast_limit=(-0.2, 0.2),p=0.5))
            augs_lst = 'no' if augs_lst == [] else augs_lst
            augs.append(augs_lst)
        df['augs_type'] = augs
        return df
    
    def resample(self,df):
        df_1 = df[df['target'] == 1].copy()
        df_0 = df[df['target'] == 0].copy()
        
        df_1 = pd.concat([df_1] * self.cfg.positive_sampling,ignore_index=True,axis=0)
        df_0 = df_0.sample(len(df_1) * self.cfg.negs_per_positive)
        
        df_1 = self.set_augs(df_1)
        df_0['augs_type'] = 'no'
        
        return pd.concat([df_0,df_1],ignore_index=True,axis=0)
    
    def prepare_data(self):
        if self.is_prepared: return None
        self.df = make_df(self.cfg.train_data)
        self.transforms = Transforms
        self.reader = h5py.File(self.cfg.train_hdf5,'r')
        self.is_prepared = True
        
    def setup(self, stage: str):
        if self.is_setup: return None
        kf = StratifiedGroupKFold(n_splits=self.cfg.nfolds, shuffle=True, random_state=self.cfg.seed)
        splits = [(x,y) for x,y in  kf.split(self.df,self.df['target'],self.df['patient_id'])][CFG.gen_fold_number]
        self.train_df, self.val_df = self.df.iloc[splits[0]], self.df.iloc[splits[1]]
        self.val_df['augs_type'] = 'no'
        kf = StratifiedGroupKFold(n_splits=self.cfg.nfolds, shuffle=True, random_state=self.cfg.seed)
        splits = [(x,y) for x,y in  kf.split(self.train_df,self.train_df['target'],self.train_df['patient_id'])][CFG.fold_number]
        self.train_df, self.predict_df = self.train_df.iloc[splits[0]], self.train_df.iloc[splits[1]]
        self.predict_df['augs_type'] = 'no'
        self.re_train_df = self.resample(self.train_df)
        self.train_dataset = PLDataset(self.re_train_df,self.transforms,self.reader,mode='train')
        self.val_dataset = PLDataset(self.val_df,self.transforms,self.reader,mode='val')
        self.predict_dataset = PLDataset(self.predict_df,self.transforms,self.reader,mode='val')
        self.is_setup = True
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset,
                         batch_size=self.cfg.batch_size,
                         num_workers=self.cfg.num_workers,
                         pin_memory=True,
                         shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.val_dataset,
                          batch_size=self.cfg.eval_batch_size,
                          num_workers=self.cfg.num_workers,
                          pin_memory=True,
                          shuffle=False)
    
    def predict_dataloader(self):
        return DataLoader(self.predict_dataset,
                          batch_size=self.cfg.eval_batch_size,
                          num_workers=self.cfg.num_workers,
                          pin_memory=True,
                          shuffle=False)

In [10]:
def p_auc_tpr(v_gt, v_pred, min_tpr=0.80, sample_weight=None):
    """Computes the area under the AUC above a minumum TPR.

    Args:
        v_gt: ground truth vector (1s and 0s)
        v_p: predictions vector of scores ranging [0, 1]
        min_tpr: minimum true positive threshold (sensitivity)

    Returns:
        Float value range [0, 1]
    """
    if len(np.unique(v_gt)) != 2:
        raise ValueError(
            "Only one class present in y_true. ROC AUC score "
            "is not defined in that case."
        )
    
    # redefine the target. set 0s to 1s and 1s to 0s
    v_gt = abs(np.asarray(v_gt)-1)
    v_pred = abs(np.asarray(v_pred)-1)
    max_fpr = abs(1-min_tpr)
    
    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=sample_weight)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)

    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)
    return(partial_auc)

In [11]:
class AverageMeter():
    def __init__(self):
        self.preds = []
        self.labels = []
        self.preds_round = []
        self.history = []
    
    def update(self,y_t,y_p,y_pr):
        self.labels += y_t
        self.preds_round += y_pr
        self.preds += y_p
        
    def clean(self):
        self.preds = []
        self.labels = []
        self.preds_round = []

    def calc_metrics(self):
        metrics = {}
        try:
            metrics['prauc'] = p_auc_tpr(self.labels,np.stack(self.preds)[:,1])
        except:
            metrics['prauc'] = -100
        self.history.append(metrics)
        return metrics

In [12]:
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.cfg = CFG.model
        self.encoder = timm.create_model(
                self.cfg.model,
                pretrained=self.cfg.pretrained,
                in_chans=self.cfg.num_chanels,
                num_classes=-1,
                drop_path_rate=self.cfg.drop_path_rate,
                global_pool=self.cfg.global_pool
        )
        self.cls_drop = nn.Dropout(self.cfg.cls_drop)
        self.fc = nn.Linear(self.cfg.hidden_size, self.cfg.num_labels)
        self._init_weights(self.fc)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=0.02)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    
    def forward(self, image, return_features=False):
        features = self.encoder(image)
        if return_features:
            return features
        logits = self.fc(self.cls_drop(features))
        return logits

In [13]:
class PLModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.cfg = CFG.model
        self.model = Model()
        self.avg_meter = AverageMeter()
        self.criterion = nn.CrossEntropyLoss()
        
    def forward(self, batch):
        output = self.model(batch['image'])
        return output

    def training_step(self, batch, i):
        logits = self(batch)
        loss = self.criterion(logits,batch['labels'])
        self.log('train_loss', loss.item())
        return loss
            
    def validation_step(self, batch, i):
        logits = self(batch)
        loss = self.criterion(logits,batch['labels'])
        self.log('val_loss',loss.item())
        preds = logits.argmax(dim=-1).tolist()
        self.avg_meter.update(batch['labels'].tolist(),logits.tolist(),preds)
    
    def predict_step(self, batch, i):
        logits = self(batch)
        return logits.softmax(dim=-1)[:,1].tolist()
                
    def on_validation_epoch_end(self):
        metrics = self.avg_meter.calc_metrics()
        self.log_dict(metrics)
        self.avg_meter.clean()
            
    def configure_optimizers(self):        
        optimizer_parameters = [
            {'params': self.model.encoder.parameters(),
             'lr': self.cfg.lr, 'weight_decay': self.cfg.weight_decay},
            {'params': self.model.fc.parameters(),
             'lr': self.cfg.lr_head, 'weight_decay': self.cfg.weight_decay_head}
        ]
        
        optim = self.cfg.optim(
            optimizer_parameters,
            lr=self.cfg.lr,
            betas=self.cfg.betas,
            weight_decay=self.cfg.weight_decay,
            eps=self.cfg.eps
        )
        
        if self.cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warmup_ratio,
                                                        num_cycles=self.cfg.num_cycles)
        elif self.cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(optim,
                                                        num_training_steps=self.cfg.num_training_steps,
                                                        num_warmup_steps=self.cfg.num_training_steps * self.cfg.warmup_ratio)
        else:
            return optim
        
        scheduler = {'scheduler': scheduler,'interval': 'step', 'frequency': 1}

        return [optim], [scheduler]

In [14]:
def train_fold(fold_gen=0,fold_n=0):
    pl.seed_everything(56)
    CFG.fold_number = fold_n
    CFG.gen_fold_number = fold_gen
    
    dm = PLDataModule()
    dm.prepare_data()
    dm.setup(0)
    
    CFG.model.num_training_steps = len(dm.train_dataloader()) * CFG.model.max_epoches
    model = PLModule()
    
    wandb.login(key="31520b01739d418e5d77a11fd8a79a70b189b8bc")
    os.environ['WANDB_API_KEY'] = "31520b01739d418e5d77a11fd8a79a70b189b8bc"
    wandb.init(project='Kaggle_ISIC',name='effnet_medium',config=set_wandb_cfg())
    
    model = PLModule()
    
    lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='step')
    checkpoint_cb = pl.callbacks.ModelCheckpoint(
        dirpath=f'./{CFG.data.output_prefix}_outputs_{fold_gen}_{fold_n}/',
        filename='model_{epoch:02d}-{prauc:.4f}',
        monitor='prauc',
        mode='max',
        save_last=True
    )

    trainer = pl.Trainer(
        accelerator="gpu",
        precision=32,
        callbacks = [lr_monitor,checkpoint_cb],
        logger = pl.loggers.WandbLogger(save_code=True),
        log_every_n_steps=1,
        min_epochs=1,
        devices=1,
        check_val_every_n_epoch=1,
        max_epochs=CFG.model.max_epoches
    )
    trainer.fit(model,dm.train_dataloader())
    preds_val = trainer.predict(model,dm.val_dataloader())
    preds_off = trainer.predict(model,dm.predict_dataloader())
    model = model.cpu()
    del model, trainer, dm, checkpoint_cb, lr_monitor
    gc.collect()
    torch.cuda.empty_cache()
    return preds_val, preds_off

In [ ]:
dm = PLDataModule()
dm.prepare_data()
dm.setup(0)

In [112]:
preds_val_00, preds_off_00 = train_fold(fold_gen=0,fold_n=0)

Seed set to 56
/tmp/ipykernel_1563/2786575331.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)
/tmp/ipykernel_1563/2697564709.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.val_df['augs_type'] = 'no'
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /notebooks/effnet_res_outputs_0_0 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Model            | 51.1 M | train
1 | criterion | CrossEntropyLoss | 0      | train
---------------

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:384: `ModelCheckpoint(monitor='prauc')` could not find the monitored key in the returned metrics: ['lr-AdamW/pg1', 'lr-AdamW/pg2', 'train_loss', 'epoch', 'step']. HINT: Did you call `log('prauc', value)` in the `LightningModule`?
`Trainer.fit` stopped: `max_epochs=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [113]:
preds_val_00

[[0.0017087602755054832,
  0.0027210162952542305,
  0.03005494549870491,
  0.004184969700872898,
  0.008842012844979763,
  0.2619289755821228,
  0.0008199719595722854,
  0.0007800693856552243,
  0.004216058179736137,
  0.0021331363823264837,
  0.0019321542931720614,
  0.01801264099776745,
  0.0058243595995008945,
  0.009426368400454521,
  0.0019109342247247696,
  0.007942215539515018,
  0.0132762361317873,
  0.002569167874753475,
  0.0051903086714446545,
  0.010283422656357288,
  0.002590342191979289,
  0.0013752130325883627,
  0.0017107061576098204,
  0.0013057583710178733,
  0.007874825038015842,
  0.0006809309707023203,
  0.06626584380865097,
  0.0004013737489003688,
  0.0034954282455146313,
  0.0008970889612101018,
  0.0036575989797711372,
  0.0029300525784492493,
  0.020004315301775932,
  0.008511029183864594,
  0.004513198044151068,
  0.001439951010979712,
  0.013839744962751865,
  0.007202572654932737,
  0.019849596545100212,
  0.0007128678262233734,
  0.0007835018332116306,
  0

In [20]:
preds_val_01, preds_off_01 = train_fold(fold_gen=0,fold_n=1)

Seed set to 56
/tmp/ipykernel_79/2786575331.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-AdamW/pg1,███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
lr-AdamW/pg2,███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train_loss,▆▂▄▃▃█▃▄▃▄▄▃▂▃▃▄▃▂▂▃▂▃▄▃▂▃▄▁▃▃▂▂▂▃▂▃▂▁▃▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,0
lr-AdamW/pg1,0.0
lr-AdamW/pg2,0.0
train_loss,0.11252
trainer/global_step,426


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /notebooks/effnet_res_outputs_0_1 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Model            | 51.1 M | train
1 | criterion | CrossEntropyLoss | 0      | train
---------------

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [21]:
preds_val_02, preds_off_02 = train_fold(fold_gen=0,fold_n=2)

Seed set to 56
/tmp/ipykernel_79/2786575331.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-AdamW/pg1,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
lr-AdamW/pg2,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train_loss,▇▂▂▂▅█▄▃▁▂▃▁▂▁█▄▄▃▂▅▄▁▅▄▁▃▂▃▁▆▃▃▆▃▄▁▁▂▅▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,0
lr-AdamW/pg1,0.0
lr-AdamW/pg2,0.0
train_loss,0.14492
trainer/global_step,429


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /notebooks/effnet_res_outputs_0_2 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Model            | 51.1 M | train
1 | criterion | CrossEntropyLoss | 0      | train
---------------

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [22]:
preds_val_03, preds_off_03 = train_fold(fold_gen=0,fold_n=3)

Seed set to 56
/tmp/ipykernel_79/2786575331.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-AdamW/pg1,███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
lr-AdamW/pg2,███████▇▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train_loss,▅▂▄▃▃█▂▅▄▃▂▂▂▃▃▂▃▁▄▃▁▂▄▄▂▂▃▂▃▁▁▂▂▃▁▂▁▁▃▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,0
lr-AdamW/pg1,0.0
lr-AdamW/pg2,0.0
train_loss,0.07049
trainer/global_step,426


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /notebooks/effnet_res_outputs_0_3 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Model            | 51.1 M | train
1 | criterion | CrossEntropyLoss | 0      | train
---------------

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [23]:
preds_val_04, preds_off_04 = train_fold(fold_gen=0,fold_n=4)

Seed set to 56
/tmp/ipykernel_79/2786575331.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr-AdamW/pg1,███████▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
lr-AdamW/pg2,███████▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
train_loss,▅▄█▅▅▂▅▄▂▂▆▆█▁▂▂▃▂▄▇▄▃▄▂▃▂▆▃▂▂▂▃▄▂▂▆▂▅▃▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch,0
lr-AdamW/pg1,0.0
lr-AdamW/pg2,0.0
train_loss,0.01422
trainer/global_step,452


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /notebooks/effnet_res_outputs_0_4 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | Model            | 51.1 M | train
1 | criterion | CrossEntropyLoss | 0      | train
---------------

Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [24]:
all_val_0 = np.mean([np.concatenate(x) for x in [preds_val_00,preds_val_01,preds_val_02,preds_val_03,preds_val_04]],axis=0)

In [25]:
off_0 = np.concatenate([np.concatenate(x) for x in [preds_off_00,preds_off_01,preds_off_02,preds_off_03,preds_off_04]],axis=0)

In [26]:
%%capture
preds_val_10, preds_off_10 = train_fold(fold_gen=1,fold_n=0)
preds_val_11, preds_off_11 = train_fold(fold_gen=1,fold_n=1)
preds_val_12, preds_off_12 = train_fold(fold_gen=1,fold_n=2)
preds_val_13, preds_off_13 = train_fold(fold_gen=1,fold_n=3)
preds_val_14, preds_off_14 = train_fold(fold_gen=1,fold_n=4)

Seed set to 56
GPU available: True (cuda), used: True
/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_run.py:2378: UserWarning: Run (4shvza1q) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /notebooks/effnet_res_outputs_1_0 e

In [27]:
all_val_1 = np.mean([np.concatenate(x) for x in [preds_val_10,preds_val_11,preds_val_12,preds_val_13,preds_val_14]],axis=0)
off_1 = np.concatenate([np.concatenate(x) for x in [preds_off_10,preds_off_11,preds_off_12,preds_off_13,preds_off_14]],axis=0)

In [28]:
%%capture
preds_val_20, preds_off_20 = train_fold(fold_gen=2,fold_n=0)
preds_val_21, preds_off_21 = train_fold(fold_gen=2,fold_n=1)
preds_val_22, preds_off_22 = train_fold(fold_gen=2,fold_n=2)
preds_val_23, preds_off_23 = train_fold(fold_gen=2,fold_n=3)
preds_val_24, preds_off_24 = train_fold(fold_gen=2,fold_n=4)

Seed set to 56
GPU available: True (cuda), used: True
/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_run.py:2378: UserWarning: Run (4shvza1q) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /notebooks/effnet_res_outputs_2_0 e

In [29]:
all_val_2 = np.mean([np.concatenate(x) for x in [preds_val_20,preds_val_21,preds_val_22,preds_val_23,preds_val_24]],axis=0)
off_2 = np.concatenate([np.concatenate(x) for x in [preds_off_20,preds_off_21,preds_off_22,preds_off_23,preds_off_24]],axis=0)

In [30]:
%%capture
preds_val_30, preds_off_30 = train_fold(fold_gen=3,fold_n=0)
preds_val_31, preds_off_31 = train_fold(fold_gen=3,fold_n=1)
preds_val_32, preds_off_32 = train_fold(fold_gen=3,fold_n=2)
preds_val_33, preds_off_33 = train_fold(fold_gen=3,fold_n=3)
preds_val_34, preds_off_34 = train_fold(fold_gen=3,fold_n=4)

Seed set to 56
GPU available: True (cuda), used: True
/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_run.py:2378: UserWarning: Run (4shvza1q) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /notebooks/effnet_res_outputs_3_0 e

In [31]:
all_val_3 = np.mean([np.concatenate(x) for x in [preds_val_30,preds_val_31,preds_val_32,preds_val_33,preds_val_34]],axis=0)
off_3 = np.concatenate([np.concatenate(x) for x in [preds_off_30,preds_off_31,preds_off_32,preds_off_33,preds_off_34]],axis=0)

In [32]:
%%capture
preds_val_40, preds_off_40 = train_fold(fold_gen=4,fold_n=0)
preds_val_41, preds_off_41 = train_fold(fold_gen=4,fold_n=1)
preds_val_42, preds_off_42 = train_fold(fold_gen=4,fold_n=2)
preds_val_43, preds_off_43 = train_fold(fold_gen=4,fold_n=3)
preds_val_44, preds_off_44 = train_fold(fold_gen=4,fold_n=4)

Seed set to 56
GPU available: True (cuda), used: True
/usr/local/lib/python3.11/dist-packages/wandb/sdk/wandb_run.py:2378: UserWarning: Run (4shvza1q) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params | Mode 
-------------------------------------------------------


In [33]:
all_val_4 = np.mean([np.concatenate(x) for x in [preds_val_40,preds_val_41,preds_val_42,preds_val_43,preds_val_44]],axis=0)
off_4 = np.concatenate([np.concatenate(x) for x in [preds_off_40,preds_off_41,preds_off_42,preds_off_43,preds_off_44]],axis=0)

In [34]:
state = {
    'all_val_0': all_val_0,
    'all_val_1': all_val_1,
    'all_val_2': all_val_2,
    'all_val_3': all_val_3,
    'all_val_4': all_val_4,
    'off_0': off_0,
    'off_1': off_1,
    'off_2': off_2,
    'off_3': off_3,
    'off_4': off_4
}

In [40]:
from copy import copy

def prepare_state(state):
    new_state = copy(state)
    dm = PLDataModule()
    dm.prepare_data()
    dm.setup(0)
    
    for i in range(5):
        kf = StratifiedGroupKFold(n_splits=dm.cfg.nfolds, shuffle=True, random_state=dm.cfg.seed)
        splits = [(x,y) for x,y in  kf.split(dm.df,dm.df['target'],dm.df['patient_id'])][i]
        train_df, val_df = dm.df.iloc[splits[0]], dm.df.iloc[splits[1]]
        predict_idx = []
        for j in range(5):
            kf = StratifiedGroupKFold(n_splits=dm.cfg.nfolds, shuffle=True, random_state=dm.cfg.seed)
            splits = [(x,y) for x,y in  kf.split(train_df,train_df['target'],train_df['patient_id'])][j]
            train_df_, predict_df = train_df.iloc[splits[0]], train_df.iloc[splits[1]]
            predict_idx.extend(predict_df.index.tolist())
        sorted_idx = np.argsort(predict_idx)
        print(len(sorted_idx),new_state[f'off_{i}'].shape[0],len(predict_idx))
        new_state[f'off_{i}'] = new_state[f'off_{i}'][sorted_idx]
    return new_state

In [41]:
new_state = prepare_state(state)

/tmp/ipykernel_306/2786575331.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


322769 322769 322769
328208 328208 328208
317860 317860 317860
314460 314460 314460
320939 320939 320939


In [42]:
np.save('prepared_state.npy',new_state)

In [45]:
427 * 32 // (11 * 5)

248

In [46]:
400 * 0.8 * 0.8

256.0